## Tools for Building Henderson's Mixed Model Equations for Multiple Trait Model

Use of Gibbs sampler to compute posterior mean of effects, $\mathbf{G}_0$ and $\mathbf{R}_0$. The program also allows for missing traits.


In [1]:
include("../../PedModule.jl/src/PedModule.jl")
using DataFrames
using Distributions

In [113]:
function Gibbs(A,x,b,nIter;outFreq=100)
    n = size(x,1)
    xMean = zeros(n)
    for iter = 1:nIter
        if iter%outFreq==0
            println("at sample: ",iter)
        end
        for i=1:n
            cVar = 1.0/A[i,i]
            cMean   = cVar*(b[i] - A[:,i]'x)[1,1] + x[i]
            x[i]    = randn()*sqrt(cVar) + cMean 
        end
        xMean += (x - xMean)/iter
    end
    return xMean
end

function Gibbs(A,x,b)
    n = size(x,1)
    for i=1:n
        cVar = 1.0/A[i,i]
        cMean   = cVar*(b[i] - A[:,i]'x)[1,1] + x[i]
        x[i]    = randn()*sqrt(cVar) + cMean 
    end
end

function GaussSeidel(A,x,b;tol=0.000001)
    n = size(x,1)
    for i=1:n
        x[i] = ((b[i] - A[:,i]'x)/A[i,i])[1,1] + x[i]
    end
    diff = sum((A*x-b).^2)
    iter = 0
    while ((diff/n > tol) & (iter<1000))
        iter += 1
        for i=1:n
            x[i] = ((b[i] - A[:,i]'x)/A[i,i])[1,1] + x[i]
        end
        diff = sum((A*x-b).^2)
        #println(iter," ",diff/n)
    end
    return x
end

function Jacobi(A,x,b,p;tol=0.000001)
    D       = diag(A)
    res     = A*x
    resid   = b-res
    tempSol = resid./D
    diff    = sum(resid.^2)
    n    = size(A,1)
    iter = 0
    while ((diff/n > tol) & (iter<1000))
        iter += 1
        x = p*tempSol + (1-p)*x
        res     = A*x
        resid   = b-res
        tempSol = resid./D + x
        diff    = sum(resid.^2)
        println(iter," ",diff/n)
    end
    return x
end


type TermStrVal
    str::AbstractString
    value::Float64
end

type TermLvlVal
    level::AbstractString
    value::Float64
end

type ModelTerm 
    iModel::Int64
    trmStr::AbstractString
    nFactors::Int64
    factors::Array{Symbol,1}
    str::Array{AbstractString,1}            # used to store the data for this term as strings
    val::Array{Float64,1}
    startPos::Int64                         # start pos in HMME
    nLevels::Int64                           
    X::SparseMatrixCSC{Float64,Int64}
    names::Array{Any,1}
end
type MME
    modelVec::Array{AbstractString,1}
    modelTerms::Array{ModelTerm,1}
    modelTermDict::Dict{AbstractString,ModelTerm}
    lhsVec::Array{Symbol,1}
    covVec::Array{Symbol,1}
    pedTrmVec::Array{AbstractString,1}
    X
    ySparse
    mmeLhs
    mmeRhs
    ped
    Gi::Array{Float64,2}
    R::Array{Float64,2}
    Ai
    mmePos::Int64
    missingPattern
    resVar
end

type ResVar
    R0::Array{Float64,2}
    RiDict::Dict{BitArray{1},Array{Float64,2}}
end   

function mkDict(a)
    aUnique = unique(a)
    d = Dict()
    names = Array(Any,size(aUnique,1))
    for (i,s) in enumerate(aUnique)
        names[i] = s
        d[s] = i
    end
    return d,names
end

function getRi(resVar::ResVar,sel::BitArray{1})
    if haskey(resVar.RiDict,sel)
        return resVar.RiDict[sel]
    end
    n = size(resVar.R0,1)
    RZ = zeros(n,n)
    RZ[sel,sel] = inv(resVar.R0[sel,sel])
    resVar.RiDict[sel] = copy(RZ)
    return RZ
end

function getTerm(trmStr,m)
    trm = ModelTerm(m,string(m)*":"*trmStr,0,[],[],[],0,0,spzeros(0,0),[])
    factorVec = split(trmStr,"*")
    trm.nFactors = length(factorVec)
    trm.factors = [symbol(strip(f)) for f in factorVec]
    return trm
end

function initMME(models::AbstractString,R::Array{Float64,2})
    # returns an MME object for muilding the mme corresponding 
    # to the input string
    if models==""
        println("modelEquation is empty\n")
        return
    end
    modelVec = split(models,[';','\n'],keep=false)
    nModels  = size(modelVec,1)
    lhsVec   = Symbol[]
    modelTerms = ModelTerm[]
    dict = Dict{AbstractString,ModelTerm}()
    for (m,model) = enumerate(modelVec)
        lhsRhs = split(model,"=")
        lhsVec = [lhsVec;symbol(strip(lhsRhs[1]))]
        rhs = strip(lhsRhs[2])
        rhsVec = split(rhs,"+")    
        mTrms = [getTerm(strip(trmStr),m) for trmStr in rhsVec]
        modelTerms = [modelTerms; mTrms]
        for (i,trm) = enumerate(modelTerms) 
            dict[trm.trmStr] = modelTerms[i]
        end 
    end
    return MME(modelVec,modelTerms,dict,lhsVec,[],[],0,0,0,0,0,Array(Float64,1,1),R,0,1,0,0)
end 

function getData(trm::ModelTerm,df::DataFrame,mme::MME)
    nObs = size(df,1)
    trm.str = Array(AbstractString,nObs)
    trm.val = Array(Float64,nObs)
    if(trm.factors[1] == :intercept)                     # from Melanie's HW
        str = fill(string(trm.factors[1]),nObs)
        val = fill(1.0,nObs)
    else
        myDf = df[trm.factors]
        if trm.factors[1] in mme.covVec
            str = fill(string(trm.factors[1]),nObs)
            val = df[trm.factors[1]]
        else
            str = [string(i) for i in df[trm.factors[1]]]
            val = fill(1.0,nObs)
        end
        for i=2:trm.nFactors
            if trm.factors[i] in mme.covVec
                str = str .* fill("*"*string(trm.factors[i]),nObs)
                val = val .* df[trm.factors[i]]
            else
                str = str .* fill("*",nObs) .* [string(j) for j in df[trm.factors[i]]]
                val = val .* fill(1.0,nObs)
            end
        end
    end
    trm.str = str
    trm.val = val
end

getFactor1(str) = [strip(i) for i in split(str,"*")][1]

function getX(trm,mme::MME)
    pedSize = 0
    nObs  = size(trm.str,1)
    if trm.trmStr in mme.pedTrmVec
        trm.names   = PedModule.getIDs(mme.ped)
        trm.nLevels = length(mme.ped.idMap)
        xj = round(Int64,[mme.ped.idMap[getFactor1(i)].seqID for i in trm.str])
    else
        dict,trm.names  = mkDict(trm.str)
        trm.nLevels     = length(dict)
        xj    = round(Int64,[dict[i] for i in trm.str]) 
    end
    xi    = (trm.iModel-1)*nObs + collect(1:nObs)
    xv    = trm.val
    if mme.ped!=0
        pedSize = length(mme.ped.idMap)
        if trm.trmStr in mme.pedTrmVec
            # This is to ensure the X matrix for 
            # additive effect has the correct number of columns
            ii = 1         # adding a zero to
            jj = pedSize   # the last column in row 1
            vv = [0.0]
            xi = [xi;ii]
            xj = [xj;jj]
            xv = [xv;vv]
        end
    end 
    #make sure X has nObs*nModels rows
    nModels = size(mme.lhsVec,1)
    xi = [xi;1;nObs*nModels]
    xj = [xj;1;1]
    xv = [xv;0;0]
    trm.X = sparse(xi,xj,xv)
    trm.startPos = mme.mmePos
    mme.mmePos  += trm.nLevels
end

function mkRi(mme::MME,df::DataFrame)
    resVar = ResVar(mme.R,Dict())
    tstMsng = !isna(df[mme.lhsVec[1]])
    for i=2:size(mme.lhsVec,1)
        tstMsng = [tstMsng !isna(df[mme.lhsVec[i]])]
    end
    mme.missingPattern = tstMsng
    n    = size(tstMsng,2)
    nObs = size(tstMsng,1)
    ii = Array(Int64,nObs*n^2)
    jj = Array(Int64,nObs*n^2)
    vv = Array(Float64,nObs*n^2)
    pos = 1
    for i=1:size(tstMsng,1)
        sel = reshape(tstMsng[i,:],n)
        Ri  = getRi(resVar,sel)
        for ti=1:n
            tii = (ti-1)*nObs + i
            for tj=1:n
                tjj = (tj-1)*nObs + i
                ii[pos] = tii
                jj[pos] = tjj
                vv[pos] = Ri[ti,tj]
                pos += 1
            end
        end         
    end
    mme.resVar = resVar
    return sparse(ii,jj,vv)
end

function getMME(mme::MME, df::DataFrame)
    mme.mmePos = 1
    for trm in mme.modelTerms
        getData(trm,df,mme)
        getX(trm,mme)
    end
    n   = size(mme.modelTerms,1)
    trm = mme.modelTerms[1]
    X   = trm.X
    for i=2:n
        trm = mme.modelTerms[i]
        X = [X trm.X]
    end
    y = convert(Array,df[mme.lhsVec[1]],0.0)
    for i=2:size(mme.lhsVec,1)
        y    = [y; convert(Array,df[mme.lhsVec[i]],0.0)] 
    end
    N  = size(y,1)
    ii = 1:N
    jj = fill(1,N)
    vv = y
    ySparse = sparse(ii,jj,vv)
    nObs = size(df,1)
    Ri = mkRi(mme,df)
    mme.X = X
    mme.ySparse = ySparse 
    mme.mmeLhs = X'Ri*X
    mme.mmeRhs = X'Ri*ySparse
    if mme.ped != 0
        ii,jj,vv = PedModule.HAi(mme.ped)
        HAi = sparse(ii,jj,vv)
        mme.Ai = HAi'HAi
        addA(mme)
    end   
end

function getNames(mme)
    names = Array(String,0)
    for trm in mme.modelTerms
        for name in trm.names
            push!(names,trm.trmStr*": "*name)
        end
    end
    return names
end  

function covList(mme::MME, covStr::AbstractString)
    covVec = split(covStr," ",false) 
    mme.covVec = [symbol(i) for i in covVec]
    nothing
end

function getSolJ(mme::MME, df::DataFrame)
    if size(mme.mmeRhs)==() 
        getMME(mme,df)
    end
    p = size(mme.mmeRhs,1)
    return [getNames(mme) Jacobi(mme.mmeLhs,fill(0.0,p),mme.mmeRhs,0.3,tol=0.000001)]
end

function getSolG(mme::MME, df::DataFrame)
    if size(mme.mmeRhs)==() 
        getMME(mme,df)
    end
    p = size(mme.mmeRhs,1)
    return [getNames(mme) GaussSeidel(mme.mmeLhs,fill(0.0,p),mme.mmeRhs,tol=0.000001)]
end

function setAsRandom(mme::MME,randomStr::AbstractString,ped::PedModule.Pedigree, G::Array{Float64,2})
    pedTrmVec = split(randomStr," ",keep=false)
    res = []
    for trm in pedTrmVec
        for (m,model) = enumerate(mme.modelVec)
            strVec  = split(model,['=','+'])
            strpVec = [strip(i) for i in strVec]
            if trm in strpVec
                res = [res;string(m)*":"*trm]
            end
        end
    end
    mme.pedTrmVec = res
    mme.ped = ped
    mme.Gi = inv(G)
    nothing
end

function addA(mme::MME)
    pedTrmVec = mme.pedTrmVec
    for (i,trmi) = enumerate(pedTrmVec)
        pedTrmi  = mme.modelTermDict[trmi]
        startPosi  = pedTrmi.startPos
        endPosi    = startPosi + pedTrmi.nLevels - 1
        for (j,trmj) = enumerate(pedTrmVec)
            pedTrmj  = mme.modelTermDict[trmj]
            startPosj  = pedTrmj.startPos
            endPosj    = startPosj + pedTrmj.nLevels - 1 
            mme.mmeLhs[startPosi:endPosi,startPosj:endPosj] = 
            mme.mmeLhs[startPosi:endPosi,startPosj:endPosj] + mme.Ai*mme.Gi[i,j] 
        end
    end
end

addA (generic function with 1 method)

In [4]:
ped = PedModule.mkPed("sim.ped");

In [5]:
dfGen = readtable("sim.gen", separator = ' ');

In [6]:
Q = convert(Array{Float64,2},dfGen[:,collect(2:end)]);
α1 = randn(200)
α2 = randn(200)
a1 = Q*α1
a2 = Q*α2;

In [7]:
D = diagm(vec(sqrt(var([a1 a2],1))'))

2x2 Array{Float64,2}:
 8.54232  0.0    
 0.0      7.87874

In [8]:
R = cor([a1 a2])

2x2 Array{Float64,2}:
 1.0        0.0126625
 0.0126625  1.0      

In [9]:
G0 = D*R*D

2x2 Array{Float64,2}:
 72.9712     0.852224
  0.852224  62.0745  

In [10]:
R0 = diagm(vec(var([a1 a2],1)))
L  = chol(R0)
e  = L*randn(2,size(Q,1))
y = [a1 a2] + e'

4099x2 Array{Float64,2}:
 -24.5167    23.2062 
   0.448859   7.72298
   0.217692  33.2118 
 -17.0455    15.2028 
 -39.5634    20.0016 
 -40.3758    17.0708 
 -20.9877    14.156  
 -25.9593     7.62797
   2.64354    5.80148
 -21.4323    17.05   
  -6.85471   26.8703 
  -2.89299    3.3677 
 -20.8675    27.5964 
   ⋮                 
 -48.8102    25.9565 
 -33.6007     4.87704
 -33.2385    13.3288 
 -20.561     17.1171 
  -6.6807    21.6124 
 -29.9627    23.1073 
 -35.1949     8.70033
 -21.6639    -3.40343
 -34.1614    26.5183 
 -25.8125    13.4753 
  -3.95796   27.5877 
 -25.6007    18.2949 

In [11]:
df2 = DataFrame(Animal = dfGen[:,1], y1=y[:,1],y2=y[:,2])

,Animal,y1,y2
1,10102,-24.516698298811768,23.206160278374895
2,10103,0.4488585845542836,7.722980642347891
3,10104,0.21769220851462556,33.21183157415616
4,10105,-17.04547538453811,15.202795538892659
5,10106,-39.56343588822591,20.00159233146701
6,10107,-40.375793202359915,17.070786131454852
7,10108,-20.987717152403544,14.156004892878286
8,10109,-25.95932330142873,7.627968965027289
9,10110,2.6435420471080953,5.801483369469098
10,10111,-21.43234074873155,17.049955533073483


In [12]:
df2[1,2] = NA

NA

In [115]:
head(df2)

,Animal,y1,y2
1,10102,NA,23.206160278374895
2,10103,0.4488585845542836,7.722980642347891
3,10104,0.21769220851462556,33.21183157415616
4,10105,-17.04547538453811,15.202795538892659
5,10106,-39.56343588822591,20.00159233146701
6,10107,-40.375793202359915,17.070786131454852


In [13]:
models = "y1 = intercept + Animal;
          y2 = intercept + Animal"
R = R0
mme = initMME(models,R)
setAsRandom(mme,"Animal", ped,G0)

In [135]:
msng = [true; false;false]
notMsng = !msng
resTest = collect(1:12)
i=1
indexRes = collect(0:3-1)*4+i
resTest[indexRes][notMsng] = 100

100

In [136]:
resTest[indexRes]

3-element Array{Int64,1}:
 1
 5
 9

In [137]:
resTest[(indexRes)[notMsng]] = 100

100

In [138]:
resTest[indexRes]

3-element Array{Int64,1}:
   1
 100
 100

In [104]:
function sampleMissingResiduals(mme,resVec)
    msngPtrn = mme.missingPattern
    n,k = size(msngPtrn)
    yIndex = collect(0:k-1)*n
    allTrue = fill(true,k)
    for i=1:n
        notMsng = reshape(msngPtrn[i,:,],k)
        if (notMsng!=allTrue)
            msng    = !notMsng
            nMsng   = sum(msng)
            resi    = resVec[yIndex+i][notMsng]
            Ri      = mme.resVar.RiDict[notMsng][notMsng,notMsng]
            Rc      = mme.R[msng,notMsng]
            L       = chol(mme.R[msng,msng] - Rc*Ri*Rc')'
            resVec[(yIndex+i)[msng]] = Rc*Ri*resi + L*randn(nMsng) 
            #resVec[yIndex+i][msng] = Rc*Ri*resi + L*randn(nMsng) this does not work!
        end
    end
end

sampleMissingResiduals (generic function with 1 method)

In [110]:
function sampleMCMC(nIter,mme,df;outFreq=100)
    getMME(mme,df2)
    p = size(mme.mmeLhs,1)
    sol = fill(0.0,p)
    solMean = fill(0.0,p)
    GaussSeidel(mme.mmeLhs,sol,mme.mmeRhs,tol=0.000001) 
    ν = 10
    nObs    = size(df,1)
    nTraits = size(mme.lhsVec,1)
    νR0 = ν + nTraits
    R0 = mme.R
    PRes = R0*(νR0 - nTraits - 1)
    SRes   = zeros(Float64,nTraits,nTraits)
    R0Mean = zeros(Float64,nTraits,nTraits)
    if mme.ped != 0
        pedTrmVec = mme.pedTrmVec
        k = size(pedTrmVec,1)
        νG0 = ν + k
        G0 = inv(mme.Gi)
        P = G0*(νG0 - k - 1)
        S = zeros(Float64,k,k)
        G0Mean = zeros(Float64,k,k)
    end
    for iter=1:nIter
        if iter%outFreq==0
            println("at sample: ",iter,"\n")
            println(G0Mean,"\n")
        end
        Gibbs(mme.mmeLhs,sol,mme.mmeRhs)
        # can make this more efficient by taking advantage of symmetry
        for (i,trmi) = enumerate(pedTrmVec)    
            pedTrmi  = mme.modelTermDict[trmi]
            startPosi  = pedTrmi.startPos
            endPosi    = startPosi + pedTrmi.nLevels - 1
            for (j,trmj) = enumerate(pedTrmVec)
                pedTrmj  = mme.modelTermDict[trmj]
                startPosj  = pedTrmj.startPos
                endPosj    = startPosj + pedTrmj.nLevels - 1
                S[i,j] = (sol[startPosi:endPosi]'*mme.Ai*sol[startPosj:endPosj])[1,1]
            end
        end
        resVec = mme.ySparse - mme.X*sol
        sampleMissingResiduals(mme,resVec)
        for traiti = 1:nTraits
            startPosi = (traiti-1)*nObs + 1
            endPosi   = startPosi + nObs - 1
            for traitj = traiti:nTraits
                startPosj = (traitj-1)*nObs + 1
                endPosj   = startPosj + nObs - 1
                SRes[traiti,traitj] = (resVec[startPosi:endPosi]'resVec[startPosj:endPosj])[1,1] 
                SRes[traiti,traitj] = SRes[traitj,traiti]
            end
        end
        R0 = rand(InverseWishart(νR0 + nObs, PRes + SRes))
        mme.R = R0
        Ri = mkRi(mme,df)
        X = mme.X
        mme.mmeLhs = X'Ri*X
        mme.mmeRhs = X'Ri*mme.ySparse
        if mme.ped != 0
            pedTrm1 = mme.modelTermDict[pedTrmVec[1]]
            q = pedTrm1.nLevels
            G0 = rand(InverseWishart(νG0 + q, P + S))
            mme.Gi = inv(G0)
            addA(mme)
        end
        solMean += (sol - solMean)/iter
        G0Mean  += (G0  - G0Mean )/iter
        R0Mean  += (R0  - R0Mean )/iter
    end
    output = Dict()
    output["posteriorMeanLocationParms"] = solMean
    output["posteriorMeanG0"] = G0Mean
    output["posteriorMeanR0"] = R0Mean
    return output
end

sampleMCMC (generic function with 1 method)

In [114]:
@time res = sampleMCMC(2000,mme,df2)
res["posteriorMeanG0"]

at sample: 100

[99.3520520270497 -0.8548131659229353
 -0.8548131659229353 58.20863955327304]

at sample: 200

[94.22184337649703 -0.1684999972359489
 -0.1684999972359489 64.78261243065077]

at sample: 300

[93.32067534561207 0.797936194989491
 0.797936194989491 70.2623725167838]

at sample: 400

[92.23290527815604 0.8970995026291004
 0.8970995026291004 72.16214986299578]

at sample: 500

[92.54138145269755 0.6959591209234381
 0.6959591209234381 72.08276241614183]

at sample: 600

[93.11493999988461 0.9741890152086364
 0.9741890152086364 71.61358784845412]

at sample: 700

[94.58460046889421 0.7856908846636945
 0.7856908846636945 71.44139918403656]

at sample: 800

[94.99175506425898 0.9545077833599203
 0.9545077833599203 71.74786239016792]

at sample: 900

[94.29843287677022 0.8017688523282259
 0.8017688523282259 72.85510336998647]

at sample: 1000

[94.06723442356129 0.9542665791998469
 0.9542665791998469 74.05077454260619]

at sample: 1100

[94.2688882527389 0.8992770077826757
 0.89

2x2 Array{Float64,2}:
 92.3069    1.33832
  1.33832  75.0051 



[92.30874811855898 1.3356496227493266
 1.3356496227493266 74.99797395033926]

150.857966 seconds (896.92 M allocations: 74.257 GB, 6.20% gc time)


In [17]:
res["posteriorMeanR0"]

2x2 Array{Float64,2}:
 80.9276     -0.0481069
 -0.0481069  69.0774   

In [18]:
p = size(mme.mmeLhs,1)
sol = fill(0.0,p)
Gibbs(mme.mmeLhs,sol,mme.mmeRhs)

In [107]:
resVec = mme.ySparse - mme.X*sol

8198x1 Array{Float64,2}:
  23.849   
   9.92661 
  12.9877  
  -0.827962
  -9.71496 
 -19.5006  
   8.23418 
 -11.9279  
  11.4829  
  -6.31739 
   7.97674 
   7.6602  
  -1.72906 
   ⋮       
   0.194921
   0.816192
   8.47903 
  -1.899   
  11.9672  
  -2.56154 
  -4.84058 
 -12.8214  
   4.44336 
   0.473818
   7.84256 
   4.86123 

In [108]:
sampleMissingResiduals(mme,resVec)

In [109]:
resVec

8198x1 Array{Float64,2}:
   0.756685
   9.92661 
  12.9877  
  -0.827962
  -9.71496 
 -19.5006  
   8.23418 
 -11.9279  
  11.4829  
  -6.31739 
   7.97674 
   7.6602  
  -1.72906 
   ⋮       
   0.194921
   0.816192
   8.47903 
  -1.899   
  11.9672  
  -2.56154 
  -4.84058 
 -12.8214  
   4.44336 
   0.473818
   7.84256 
   4.86123 